In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

[    0.00] Initializing mpi-sppy


In [2]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 48  # Number of execution scenarios                     (chi)

M = 1e3  # Big M 

In [3]:
def build_model(cap_factor):
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=125, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system')
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    # com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    # prod_max = {0:30, 1: 60, 2: 90, 3: 120, 4: 150}
    # prod_min = {0:0, 1: 30, 2: 60, 3: 90, 4: 120}
    # rate_max = {0:1.25/2, 1: 1/2, 2: 0.75/2, 3: 0.5/2, 4: 0.25/2}
    # mode_ramp = {(0,1): 5, (1,2): 5}
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 125
    
    com1_procure = Process(name='procure com1', prod_max=com1_process_capacity, conversion={com1_pur: -1, com1_in: 1}, capex=0.01, vopex=0.01, prod_min=0.01, label='Procure com1', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.01, vopex=0.01, prod_min=0.01, label='Sell com1')
    com1_opt_procure = Process(name='procure optional com1', prod_max=75, conversion={com1_pur: -1, com1_in:1}, capex=10, vopex=0.1, prod_min=0.01, label='Procure optional com1')
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=0.01, label='Commodity 1 received from location 1')
    # com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Commodity 1 received from location 2')
    
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Process the commodity through the location')
    # com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={0:{com1_in: -1, com1_out: 1}, 1:{com1_in: -1, com1_out: 1}, 2:{com1_in: -1, com1_out: 1}, 3:{com1_in: -1, com1_out: 1}, 4:{com1_in: -1, com1_out: 1}},  capex=0.01, vopex=0.01, prod_min=0.01, label='Process the commodity through the location')
    # 
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.01, vopex=0.01, prod_min=0.01, label='Process the commodity through the location')
    
    com1_store10 = Process(name='com1_store10', prod_max=com1_process_capacity, capex=0.01, vopex=0.1, storage_capex=10, store_min=0.01, store_max= 40, prod_min=0.01, label="Storage capacity of 10 units", storage=com1_in, storage_cost=0.02)
    com1_store20 = Process(name='com1_store20', prod_max=com1_process_capacity, capex=0.01, vopex=0.2, storage_capex=20, store_min=0.01, store_max= 80, prod_min=0.01, label="Storage capacity of 20 units", storage=com1_in, storage_cost=0.02)
    com1_store50 = Process(name='com1_store50', prod_max=com1_process_capacity, capex=0.01, vopex=0.5, storage_capex=50, store_min=0.01, store_max= 200, prod_min=0.01, label="Storage capacity of 50 units", storage=com1_in, storage_cost=0.02)
    
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.01, vopex=0.01, prod_min=0.01, label='Send commodity one from location 1')
    # com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.01, vopex=0.01, prod_min=com1_process_capacity, label='Send commodity one from location 2')
    
    
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_process, com1_store10, com1_store20, com1_store50, com1_loc1_send, com1_sell, com1_opt_procure}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, capacity_factor={com1_procure: cap_factor[['com1_procure']]})
    
   
    locset = [loc1]



    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = 100
    demand_penalty = 50
    
    demand_dict = {i: {com1_sold: daily_demand} if i == loc1 else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty} if i == loc1 else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario_baseline', network= loc1, scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Scenario with perfect information')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================
    
    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)
    
    
    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    
    problem_mincost.first_stage_cost  = capex_process
    
    return scenario, problem_mincost

In [27]:
#Expected Cost under Perfect Information
all_scenario_names = ["good", "average", "bad"]
ns_dict = {'good': 48, "average": 24, "bad":12}
scenario_probabilities = {'good': 0.33, 'average': 0.33, 'bad':0.34} 

exCost_PI = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen_PI, model_PI = build_model(cap_factor=cap_factor)
    # for loc in model.locations:
    #     model.X_P[loc,"procure com1",scen.network_scale_level].fixed= True
    #     model.X_P[loc,"procure com1",scen.network_scale_level].value= 1
    solver = SolverFactory("gurobi")
    solver.solve(model_PI)
    
    # model_PI.X_S.pprint()
    # model_PI.X_P.pprint()
    # model_PI.Cap_P.pprint()
    # model_PI.Cap_S.pprint()
    # model_PI.Capex_process.pprint()
    
    print(value(model_PI.objective_cost_w_demand_penalty))
    
    exCost_PI += value(model_PI.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
objective cost w demand penalty
1011.0
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penal

In [5]:
print(f"Expected cost under perfect information: {exCost_PI:.3f}")

Expected cost under perfect information: 176165.146


In [6]:
def scenario_creator(scenario_name):
    if scenario_name=='good':
        ns = 48
    elif scenario_name=='average':
        ns = 24
    elif scenario_name=='bad':
        ns = 12
    else:
        raise ValueError("Unrecognized scenario name")
    
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns + [0]*(_exec_scenarios-ns)})
    scen, model = build_model(cap_factor=cap_factor)
    sputils.attach_root_node(model, model.first_stage_cost, [model.X_P, model.Cap_P, model.X_S, model.Cap_S])
    model._mpisppy_probability = scenario_probabilities[scenario_name]
    return model

In [7]:
from mpisppy.opt.ef import ExtensiveForm

options = {"solver": "gurobi"}
all_scenario_names = ["good", "average", "bad"]
ef_UI = ExtensiveForm(options, all_scenario_names, scenario_creator, model_name='model_UI')
results = ef_UI.solve_extensive_form()

[    3.53] Initializing SPBase
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
objective cost w demand penalty
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility

In [8]:
exCost_UI = ef_UI.get_objective_value()
print(f"Expected cost under uncertainty: {exCost_UI:.3f}")

Expected cost under uncertainty: 177087.197


In [9]:
EVPI = exCost_UI - exCost_PI
EVPI

922.0510971251351

In [10]:
ef_UI.ef.average.Inv.pprint()

Inv : Resource Inventory
    Size=1008, Index=average.locations*average.resources_store*average.scales_scheduling
    Key                                               : Lower : Value              : Upper : Fixed : Stale : Domain
     ('loc1', 'com1_store10_com1_in_stored', 0, 0, 0) :     0 :                0.0 :  None : False : False : NonNegativeReals
     ('loc1', 'com1_store10_com1_in_stored', 0, 0, 1) :     0 :                0.0 :  None : False : False : NonNegativeReals
     ('loc1', 'com1_store10_com1_in_stored', 0, 0, 2) :     0 :                0.0 :  None : False : False : NonNegativeReals
     ('loc1', 'com1_store10_com1_in_stored', 0, 0, 3) :     0 :                0.0 :  None : False : False : NonNegativeReals
     ('loc1', 'com1_store10_com1_in_stored', 0, 0, 4) :     0 :                0.0 :  None : False : False : NonNegativeReals
     ('loc1', 'com1_store10_com1_in_stored', 0, 0, 5) :     0 :                0.0 :  None : False : False : NonNegativeReals
     ('loc1', 

In [11]:
ef_UI.ef.average.Cap_P.pprint()

Cap_P : Process Capacity
    Size=11, Index=average.locations*average.processes*average.scales_network
    Key                                   : Lower : Value              : Upper : Fixed : Stale : Domain
            ('loc1', 'com1_loc1_send', 0) :     0 :                0.0 :  None : False : False : NonNegativeReals
              ('loc1', 'com1_process', 0) :     0 :              100.0 :  None : False : False : NonNegativeReals
              ('loc1', 'com1_store10', 0) :     0 :                8.0 :  None : False : False : NonNegativeReals
    ('loc1', 'com1_store10_discharge', 0) :     0 : 1.0256410256410255 :  None : False : False : NonNegativeReals
              ('loc1', 'com1_store20', 0) :     0 :               17.0 :  None : False : False : NonNegativeReals
    ('loc1', 'com1_store20_discharge', 0) :     0 : 1.4545454545454544 :  None : False : False : NonNegativeReals
              ('loc1', 'com1_store50', 0) :     0 :                0.0 :  None : False : False : NonNegativeR

In [12]:
ef_UI.get_root_solution()

{'X_P[loc1,com1_loc1_send,0]': 0.0,
 'X_P[loc1,com1_process,0]': 1.0,
 'X_P[loc1,com1_store10,0]': 1.0,
 'X_P[loc1,com1_store10_discharge,0]': 1.0,
 'X_P[loc1,com1_store20,0]': 1.0,
 'X_P[loc1,com1_store20_discharge,0]': 1.0,
 'X_P[loc1,com1_store50,0]': 0.0,
 'X_P[loc1,com1_store50_discharge,0]': 0.0,
 'X_P[loc1,procure com1,0]': 1.0,
 'X_P[loc1,procure optional com1,0]': 1.0,
 'X_P[loc1,sell com1,0]': 1.0,
 'Cap_P[loc1,com1_loc1_send,0]': 0.0,
 'Cap_P[loc1,com1_process,0]': 100.0,
 'Cap_P[loc1,com1_store10,0]': 8.0,
 'Cap_P[loc1,com1_store10_discharge,0]': 1.0256410256410255,
 'Cap_P[loc1,com1_store20,0]': 17.0,
 'Cap_P[loc1,com1_store20_discharge,0]': 1.4545454545454544,
 'Cap_P[loc1,com1_store50,0]': 0.0,
 'Cap_P[loc1,com1_store50_discharge,0]': 0.0,
 'Cap_P[loc1,procure com1,0]': 125.0,
 'Cap_P[loc1,procure optional com1,0]': 75.0,
 'Cap_P[loc1,sell com1,0]': 100.0,
 'X_S[loc1,com1_store10_com1_in_stored,0]': 1.0,
 'X_S[loc1,com1_store20_com1_in_stored,0]': 1.0,
 'X_S[loc1,com1_st

In [13]:
# soln = ef_UI.get_root_solution()
# for (var_name, var_val) in soln.items():
#     if var_val!=0:
#         print(var_name, var_val)

In [14]:
n_avg = 24
cap_factor_avg = pandas.DataFrame(data={'com1_procure': [1]*n_avg + [0]*(_exec_scenarios-n_avg)})
scen_avg, model_avg = build_model(cap_factor=cap_factor_avg)

solver_avg = SolverFactory("gurobi")
solver_avg.solve(model_avg)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
objective cost w demand penalty


{'Problem': [{'Name': 'x1', 'Lower bound': 208895.75555555336, 'Upper bound': 208895.75555555554, 'Number of objectives': 1, 'Number of constraints': 12229, 'Number of variables': 9863, 'Number of binary variables': 14, 'Number of integer variables': 14, 'Number of continuous variables': 9849, 'Number of nonzeros': 34020, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.03600001335144043', 'Error rc': 0, 'Time': 0.22107529640197754}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [19]:
def fix_variables(model1, model2):
    model2.del_component(model2.X_P), model2.del_component(model2.Cap_P), model2.del_component(model2.X_S), model2.del_component(model2.Cap_S)
    
    model2.X_P = Var(list(model1.X_P.keys()), domain=Binary)
    model2.Cap_P = Var(list(model1.Cap_P.keys()), domain=NonNegativeReals)
    model2.X_S = Var(list(model1.X_S.keys()), domain=Binary)
    model2.Cap_S = Var(list(model1.Cap_S.keys()), domain=NonNegativeReals)
    
    def fix(var1, var2):
        for i in list(var1.keys()):
            var2[i].fixed = True
            var2[i] = value(var1[i])
            
    fix(model1.X_P, model2.X_P)
    fix(model1.Cap_P, model2.Cap_P)
    fix(model1.X_S, model2.X_S)
    fix(model1.Cap_S, model2.Cap_S)
    
    # for i in list(model1.X_P.keys()):
    #     model2.X_P[i].fixed = True
    #     model2.X_P[i] = value(model1.X_P[i])
    #     
    # for i in list(model1.Cap_P.keys()):
    #     model2.Cap_P[i].fixed = True
    #     model2.Cap_P[i] = value(model1.Cap_P[i])
    #     
    # for i in list(model1.X_S.keys()):
    #     model2.X_S[i].fixed = True
    #     model2.X_S[i] = value(model1.X_S[i])
    #     
    # for i in list(model1.Cap_S.keys()):
    #     model2.Cap_S[i].fixed = True
    #     model2.Cap_S[i] = value(model1.Cap_S[i])

In [28]:
#Value of Stochastic Solution
# all_scenario_names = ["good", "average", "bad"]
# ns_dict = {'good': 48, "average": 24, "bad": 12}
exCost_FD = 0

for scenario_name in all_scenario_names:
    cap_factor = pandas.DataFrame(data={'com1_procure': [1]*ns_dict[scenario_name] + [0]*(_exec_scenarios-ns_dict[scenario_name])})
    scen_VSS, model_VSS = build_model(cap_factor=cap_factor)
    
    fix_variables(model1=model_avg, model2=model_VSS)

    solver = SolverFactory("gurobi")
    solver.solve(model_VSS)
    
    model_VSS.X_S.pprint()
    model_VSS.X_P.pprint()
    model_VSS.Cap_S.pprint()
    model_VSS.Cap_P.pprint()
    model_VSS.Vopex_network.pprint()
    
    print(value(model_VSS.objective_cost_w_demand_penalty))

    exCost_FD += value(model_VSS.objective_cost_w_demand_penalty)*scenario_probabilities[scenario_name]

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint storage cost
constraint storage capex
constraint storage cost location
constraint storage cost network
constraint production mode
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand penalty
objective cost w demand penalty
X_S : Size=3, Index={('loc1', 'com1_store10_com1_in_stored', 0), ('loc1', 'com1_store20_com1_in_stored', 0), ('loc1', 'com1_store50_com1_in_stored', 0)}
    Key                                        : Lower : Value : Upper : Fixed : Stale : Domain
    ('loc1', 'com1_store10_com1_in_stored', 0) :     0 :   1.0 :     1 :  True :  True : Binary
    ('loc1', 'com1_store20_com1_in_stored', 0) :     0 :   1.0 :     1 :  True :  True : Binary
    ('loc1', 'com1_store50_com1_in_stored', 0

In [21]:
exCost_PI

176165.1462222222

In [22]:
exCost_UI

177087.19731934732

In [23]:
exCost_FD

176165.1462222222

In [24]:
EVPI

922.0510971251351

In [25]:
VSS = exCost_FD - exCost_UI
VSS

-922.0510971251351